# 誤差逆伝播法

* 前章で学んだ数値微分はシンプルで実装が簡単だが、計算に時間がかかるという難点がある
* 誤差逆伝播法は、重みパラメータの勾配の計算を効率良く行う手法である

## 1.  連鎖律

逆方向の伝播では「局所的な微分」を順方向とは逆方向に伝達します。  
この「局所的な微分」を伝達する原理を連鎖律と呼びます。

連鎖律とは、合成関数の微分についての性質であり、以下のように定義されている。   

ある関数が合成関数で表される場合、その合成関数の微分は、   
合成関数を構成するそれぞれの関数の微分の積によって表すことが出来る。

## 2.リンゴの例

下記の図は「100円(消費税は10%)のりんごを2個買う」という問題を計算グラフで表している。

順伝播：計算グラフの出発点から終着点への伝播   
逆伝播：順伝播の逆の伝播

<img src="./img/apple.png" title="リンゴ2個の買い物" width="800" height="500" />

In [5]:
from layer_naive import *

apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("apple_price:", apple_price)
print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

apple_price: 200
price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


## 誤差逆伝播法の勾配確認

In [4]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# ネットワークの定義
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 各重みの絶対誤差の平均を求める
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:3.849953962118686e-10
b1:2.196574470314462e-09
W2:5.659533362690474e-09
b2:1.3925124024011248e-07


## 逆誤差伝播法を使った学習

ミニバッチで繰り返し学習（重み、バイアスの更新）

In [6]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# ネットワークの定義
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 誤差逆伝播法によって、勾配を求める
    grad = network.gradient(x_batch, t_batch)
    
    # 更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.11653333333333334 0.1147
0.8990666666666667 0.9015
0.92235 0.926
0.9344166666666667 0.9369
0.9441666666666667 0.9429
0.9502166666666667 0.9483
0.9562333333333334 0.953
0.9623166666666667 0.9602
0.9635333333333334 0.9594
0.9681333333333333 0.9644
0.9700666666666666 0.9651
0.9720666666666666 0.9661
0.9741333333333333 0.9688
0.9754833333333334 0.9675
0.9752666666666666 0.9695
0.9775333333333334 0.9706
0.97955 0.9707
